In [1]:
import requests
PROMETHEUS='http://prometheus.local'


In [2]:
def wait_for_experiment_state_change():
    while True:
        response=requests.get(PROMETHEUS + '/api/v1/query', params={'query': 'max_over_time(k6_checks[5s])'}) 
        k = response.json()
        if len(k.get('data').get('result')) == 0:
            print('experiment ended')
            break;


In [3]:
def wait_for_failure():
    while True:
        response=requests.get(PROMETHEUS + '/api/v1/query', params={'query': 'max_over_time(k6_http_req_failed[5s])'}) 
        k = response.json()
#         print(k)
        if len(k.get('data').get('result')) != 1:
            response=requests.get(PROMETHEUS + '/api/v1/query', params={'query': 'sum_over_time(k6_http_req_failed[1m])'})
            k = response.json()
            print(k)
            types = list(map(lambda x: x.get('metric').get('status'), k.get('data').get('result')))
            print(types)
            non200s = list(filter(lambda x: x != '200', types))
            print(non200s)
            if len(non200s) > 0:
                print('run failed')
                return True
            else :
                print('run success')
                return False

In [4]:
def reset_experiment():
    r = requests.get('http://tester.local/reset')
    print(r.text)

In [5]:
def start_experiment():
    r = requests.get('http://tester.local/start')
    print(r.text)

In [6]:
def get_avg_latency():
    response=requests.get(PROMETHEUS + '/api/v1/query', params={'query': 'avg_over_time(k6_custom_target_highmem_duration_avg[3m])'}) 
    print(response.json())
    k = response.json().get('data').get('result')
    print(k)
    if len(k) == 0:
        return None
    else:
        return k[0].get('value')[1]

In [8]:
print("start 3 iterations")
for i in range(3):
    reset_experiment();
    wait_for_experiment_state_change()
    print("no experiment running")

    start_experiment()
    run_failed = wait_for_failure()
    if run_failed:
        print("experiment failed")
    else: 
        latency = get_avg_latency();
        print("latency: {}".format(latency))
    
    reset_experiment();
    wait_for_experiment_state_change()
    print("experiment stopped")
print("done")

start 3 iterations
Reset done
experiment ended
no experiment running
Started
{'status': 'success', 'data': {'resultType': 'vector', 'result': [{'metric': {'error': 'EOF', 'error_code': '1000', 'expected_response': 'false', 'method': 'GET', 'run': '2022-07-21_02-03-24', 'scenario': 'target_highmem', 'status': '0', 'url': 'http://target-service.target.svc.cluster.local/highmem?count=15'}, 'value': [1658369014.864, '0.4074074074074074']}, {'metric': {'error': 'dial: connection refused', 'error_code': '1212', 'expected_response': 'false', 'method': 'GET', 'run': '2022-07-21_02-03-24', 'scenario': 'target_highmem', 'status': '0', 'url': 'http://target-service.target.svc.cluster.local/highmem?count=15'}, 'value': [1658369014.864, '11.917625466253881']}, {'metric': {'error': 'read: connection reset by peer', 'error_code': '1220', 'expected_response': 'false', 'method': 'GET', 'run': '2022-07-21_02-03-24', 'scenario': 'target_highmem', 'status': '0', 'url': 'http://target-service.target.svc.cl